## data

In [2]:
import pandas as pd
df_hr = pd.read_csv('./csv/hospital_reviews.csv')
df_hr_text = df_hr[['text']].dropna()
df_khr = pd.read_csv('./csv/Seleniums_kakao_hospital_reviews_modified.csv')
df_khr.dropna(inplace=True)

In [3]:
df_hr_text[0:500]

,text
1,원장님 진료 잘봐주십니다 경력도 오래되신걸로 알고 있고 무엇보다 친절하심다
2,원장님 경력이 오래되셨다고 지인이 추천해줘서 방문했는데요 원장님 친절하시고 진료를 ...
3,통증은 꾸준하게 관리를 해줘야 한다고 해서 몇번 방문을 했는데 여기가 제일 괜찮아서...
4,직원분들 친친절하시고 원장님 진료 잘 봐주시네요 또 방문하려고요
5,친절하세요
...,...
498,좋아요
499,친절하셔서 편안히 치료받습니다.
500,내부 시설이 좋아요
501,친절하세요


In [4]:
df_khr_extract_sec = df_khr.query('positive != 1')[['text', 'positive']]

In [5]:
df_khr_extract_sec

,text,positive
0,불친절!! 전공의 복귀해!,0.0
1,Very crowded and the doctors can be very unfri...,0.0
3,여 기 진 짜 별로임\n애기 발에 고름이 꽉차서 누렇고 퉁퉁부어있는데\n발 보더니 ...,0.0
5,고대\n구로병원에 가시거든 다른 병원가세요\n저희 아버지 일반 병실에 계시다가\n중...,0.0
6,여기 서류 처리 때문에 전화 했었는데 늙은 사람이 전화 받는 태도 말투가 기본도 안...,0.0
...,...,...
1030,최고에요~!,2.0
1031,여기는 찐입니다.~!!\n회사 지인 소개로 갔는데\n원장 선생님 진짜 명의라고 말할...,2.0
1032,2년동안 여기저기 다니며 주사 맞았는데 나르샤에서 주사 맞고 고통이 팍 줄었어요.\...,2.0
1033,"정확한 판단과 적절한 치료, 친절한 의료진",2.0


In [6]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
data = {
    'text': ['쓰레기', '쓰레기병원', '개별로임'],
    'Value': [0,0,0]
}
df_khr_extract_sec.append(data, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
df_khr_extract

## tokenizing

In [ ]:
!pip install python-mecab-ko

In [ ]:
from mecab import MeCab
mecab = MeCab()

In [ ]:
mecab.pos(' 👍 👍 👍')


In [ ]:
import re
def regex_func(text) :
    patterns = '\\\\n'
    text_regex = re.sub(pattern=patterns,repl=' ',string=text)
    return text_regex
df_khr_extract_sec['text'] = df_khr_extract_sec['text'].apply(regex_func)
df_khr_extract_sec['text']

In [ ]:
mecab.pos(df_khr_extract_sec['text'].loc[495])

In [ ]:
def extract(series) :
    morphs_nouns_list = []
    for sentence in series:
        morphs_nouns = ''
        for morph, tag in mecab.pos(sentence) : 
            if tag in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP', 'NR', 'VA', 'MAG', 'SY'] :
                morphs_nouns += (morph+' ')            
        morphs_nouns_list.append(morphs_nouns)
        pass
    return morphs_nouns_list
morphs_nouns_list = extract(df_khr_extract_sec['text'])

In [ ]:
df_khr_extract_sec['text_tokenized'] = morphs_nouns_list

In [ ]:
df_khr_extract_sec

In [ ]:
df_khr_extract_sec

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [ ]:
tfidfVectorizer.fit(df_khr_extract_sec['text_tokenized'])

In [ ]:
features = df_khr_extract_sec['text_tokenized']
target = df_khr_extract_sec['positive']

In [ ]:
tokenized_features = tfidfVectorizer.transform(features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(tokenized_features, target, test_size=0.3, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

In [ ]:
model.fit(features_train,target_train)

In [ ]:
model.predict(features_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_predict = model.predict(features_test)
print(classification_report(target_test, target_predict))

In [ ]:
# make predict function
def sample_predict(features):
    tokenized_features = tfidfVectorizer.transform(extract(features))
    return model.predict(tokenized_features)
sample_predict(['가지마세요'])